## Rule-based pipeline using SpaCy

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
from utils import read_corpus, text_processor

In [3]:
corpus = read_corpus()

In [4]:
for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 94
A2: 62
B: 110
B1: 42


In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
urls = [
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish1000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish1001-2000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish2001-3000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish3001-4000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish4001-5000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish5001-6000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish6001-7000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish7001-8000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish8001-9000",
    "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish9001-10000",
]

In [9]:
df_list = []
for url in urls:
    soup = BeautifulSoup(urlopen(url), "lxml")
    table = soup.find("tbody")
    for row in table.find_all("tr")[1:]:
        row = row.text.lower().strip().split("\n")
        df_list.append(list(filter(lambda e: e != "", row))[1:])
df = pd.DataFrame(df_list, columns = ["word", "occurrences", "lemma"])
print(df)

              word occurrences         lemma
0              que       32894           que
1               de       32116            de
2               no       29897            no
3                a       22313             a
4               la       21127            la
...            ...         ...           ...
9995         enoja           5        enojar
9996        empujó           5       empujar
9997        dilema           5        dilema
9998  desconectado           5  desconectado
9999     deprimida           5     deprimido

[10000 rows x 3 columns]


In [10]:
def flatten(list_of_sents):
    return [tok for sent in list_of_sents for tok in sent]

In [5]:
# TODO: preprocessor does not work on Aventura text for some reason

In [162]:
ex = corpus["A2"][4]["content"]
# print(ex)
# print("\n\n")
processed_ex = text_processor(ex)
print(processed_ex.text)
# print("\n\n")
# print(processed_ex.tokens)
# print("\n\n")
# print(processed_ex.lemmas)
# print("\n\n")
# print(processed_ex.tags)
# print("\n\n")
# print(processed_ex.parses)

Viendo, pues, que, en efeto, no podía menearse, acordó de acogerse a su ordinario remedio, que era pensar en algún paso de sus libros; y trújole su locura a la memoria aquel de Valdovinos y del marqués de Mantua, cuando Carloto le dejó herido en la montiña, historia sabida de los niños, no ignorada de los mozos, celebrada y aun creída de los viejos; y, con todo esto, no más verdadera que los milagros de Mahoma. Ésta, pues, le pareció a él que le venía de molde para el paso en que se hallaba; y así, con muestras de grande sentimiento, se comenzó a volcar por la tierra y a decir con debilitado aliento lo mesmo que dicen decía el herido caballero del bosque: —¿Dónde estás, señora mía, que no te duele mi mal? oh no lo sabes, señora, oh eres falsa y desleal. Y, desta manera, fue prosiguiendo el romance hasta aquellos versos que dicen: —¡Oh noble marqués de Mantua, mi tío y señor carnal! Y quiso la suerte que, cuando llegó a este verso, acertó a pasar por allí un labrador de su mesmo lugar y

In [163]:
index = df.index
ranked_tokens = []
for sent in processed_ex.tokens:
    ranked_sent = []
    for tok in sent:
        condition = df["word"] == tok
        rank = index[condition].tolist()
        rank = rank[0] + 1 if rank else 0
        ranked_sent.append(rank)
    ranked_tokens.append(ranked_sent)
flattened = flatten(ranked_tokens)
print(f"{sum(flattened)/len(flattened):.2f}")

507.37


In [164]:
index = df.index
ranked_lemmas = []
for sent in processed_ex.lemmas:
    ranked_sent = []
    for lem in sent:
        condition = df["lemma"] == lem
        rank = index[condition].tolist()
        rank = rank[0] + 1 if rank else 0
        ranked_sent.append(rank)
    ranked_lemmas.append(ranked_sent)
flattened = flatten(ranked_lemmas)
print(f"{sum(flattened)/len(flattened):.2f}")

435.76
